# 1.Setup

In [1]:
import utils_text
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print(tf.__version__)

2.4.1


In [2]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

# 2.将文本加载到数据集中

## 数据下载

In [3]:
DIRECTORY_URL = "https://storage.googleapis.com/download.tensorflow.org/data/illiad/"
FILE_NAMES = ["cowper.txt", "derby.txt", "butler.txt"]
all_text_paths = utils_text.get_text_paths(data_url = DIRECTORY_URL, file_name = FILE_NAMES)
all_text_paths

['/Users/zfwang/.keras/datasets/cowper.txt',
 '/Users/zfwang/.keras/datasets/derby.txt',
 '/Users/zfwang/.keras/datasets/butler.txt']

## 数据加载

In [4]:
all_labeled_data = utils_text.get_labeled_data(parent_dir = os.path.dirname(all_text_paths[0]), file_names = FILE_NAMES, buffer_size = BUFFER_SIZE)
all_labeled_data

<ShuffleDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [5]:
for ex in all_labeled_data.take(5):
    print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"Below the brain; his teeth were shatter'd all;">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Once deem'd effeminate, but dragging now">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Achilles! such I made thee. For with me,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'THE fight between Trojans and Achaeans was now left to rage as it'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'The flying steeds were dragging towards the ships;'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


# 3.将本文编码成数字

## 建立词汇表

In [6]:
vocabulary_set = utils_text.build_token_set(all_labeled_data)
vocabulary_size = len(vocabulary_set)

vocabulary size is 17178


## 样本编码

In [8]:
all_encoded_data = all_labeled_data.map(utils_text.encode_map_fn, vocabulary_set = vocabulary_set)

TypeError: map() got an unexpected keyword argument 'vocabulary_set'

# 分割训练数据、验证数据、测试数据

In [53]:
train_data, test_data = utils_text.split_train_test(all_encoded_data = all_encoded_data, take_size = TAKE_SIZE, buffer_size = BUFFER_SIZE, batch_size = BATCH_SIZE)
sample_text, sample_labels = next(iter(test_data))
print(sample_text[0], sample_labels[0])

tf.Tensor(
[ 2584  3838  7022  6994  3742 12016   879     0     0     0     0     0
     0     0     0], shape=(15,), dtype=int64) tf.Tensor(1, shape=(), dtype=int64)


# 建立基于 LSTM 的分类模型

In [ ]:
# 模型构建
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabulary_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64, 64]:
    model.add(tf.keras.layers.Dense(units, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation = "softmax"))

# 模型编译
model.compile(
    optimizer = "adam", 
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

# 模型训练
model.fit(train_data, epochs = 3, validation_data = test_data)

# 模型评估
test_loss, test_accuracy = model.evaluate(test_data)
print(f"\nEval loss: {test_loss}, Eval accuracy: {test_accuracy}")

Epoch 1/3
